In [25]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import linprog
import scipy.linalg
import sympy
from fractions import Fraction
import fractions
import math
from IPython.display import display
from collections import defaultdict
from warnings import warn

In [13]:
def lpgen(m, n, seed=None):
    rng = np.random.default_rng(seed)
    x = rng.uniform(size=n)
    A = rng.normal(size=(m, n))
    b = A @ x
    A = A * np.sign(b).reshape(-1, 1)
    b = b * np.sign(b)
    assert np.alltrue(x >= 0)
    assert np.alltrue(b >= 0)
    assert np.allclose(A @ x, b)
    return A, b

In [14]:
m, n = 3, 5
A_eq, b_eq = lpgen(m, n)
c = np.random.normal(size=n)
print(c)
linprog(c, A_eq=A_eq, b_eq=b_eq)

[-1.36532618  0.65575541  0.74573526 -1.59044281 -0.06465185]


     con: array([ 9.91127180e-10, -1.33021830e-09, -6.23048474e-09])
     fun: -0.6942160381575284
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([9.68980342e-01, 1.02257554e+00, 7.33154014e-10, 1.94010533e-10,
       6.46559373e-01])

In [15]:
seed = None
rng = np.random.default_rng(seed)

m, n = 4, 5

for i in range(1000):
    A = rng.integers(-5, high=6, size=(m, n))
    u = scipy.linalg.null_space(A).squeeze()
    if np.alltrue(u > 0):
        break

x = np.ones(n)
b = A @ x

In [16]:
for i in range(100):
    c_unbounded = rng.integers(-10, high=10, size=n)
    result = linprog(c_unbounded, A_eq=A, b_eq=b)
    if result.status == 3:
        break
print(f"Broke on {i}.")
print("c = ", c_unbounded)
print(np.diff([c_unbounded.dot(x + k*u) for k in range(5)]))
print(result)

Broke on 4.
c =  [ 3 -9 -5  7 -1]
[-2.89431712 -2.89431712 -2.89431712 -2.89431712]
     con: array([ 6.40869141e-04, -1.52587891e-05,  6.71386719e-04,  1.00708008e-03])
     fun: -291859470639.85376
 message: 'The algorithm terminated successfully and determined that the problem is unbounded.'
     nit: 4
   slack: array([], dtype=float64)
  status: 3
 success: False
       x: array([2.20711435e+10, 1.89309401e+10, 8.87331744e+10, 3.75029999e+10,
       6.54956697e+09])


In [17]:
for i in range(100):
    c_success = rng.integers(-10, high=10, size=n)
    result = linprog(c_success, A_eq=A, b_eq=b)
    if result.status == 0:
        break
print(f"Broke on {i}.")
print(c_success)
print(result)

x = result.x
x_frac = [Fraction(xi).limit_denominator(100000) for xi in result.x]
x_float = np.array([float(xi) for xi in x_frac])
assert np.allclose(x, x_float)
print(x_frac)

Broke on 1.
[ 3 -2  7 -8 -5]
     con: array([-1.06581410e-14, -1.77635684e-15,  1.77635684e-15,  2.66453526e-15])
     fun: -8.569261880679264
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([7.51263903e-01, 7.86653185e-01, 2.32366274e-12, 5.77350859e-01,
       9.26188069e-01])
[Fraction(743, 989), Fraction(778, 989), Fraction(0, 1), Fraction(571, 989), Fraction(916, 989)]


In [18]:
def make_unbounded_polyhedron(m, n, rng=None, seed=None, max_iter=1000):
    if rng is None:
        rng = np.random.default_rng(seed)

    for i in range(max_iter):
        A = rng.integers(-9, high=10, size=(m, n))
        AA = sympy.Matrix(A)
        if AA.rank() < m:
            continue
        u = AA.nullspace()[0]
        if all([u[i, 0] > 0 for i in range(n)]):
            print(i)
            print(u)
            u_den = [u[i, 0].denominator for i in range(n)]
            d = math.lcm(*u_den)
            u = np.array([int(d*u[i, 0]) for i in range(n)])
            assert np.allclose(A @ u, 0)
            assert np.alltrue(u > 0)
            break
        if i == max_iter - 1:
            raise Exception("max_iter reached.")
    
    b = A @ np.ones(n, dtype=int)
    s = np.sign(b + 1e-10).astype(int)
    A *= s.reshape(-1, 1)
    b *= s
    
    return A, b, u

def make_c(A, b, u=None, status=0, rng=None, seed=None, max_iter=1000):
    m, n = A.shape

    if rng is None:
        rng = np.random.default_rng(seed)

    for i in range(max_iter):
        c = rng.integers(-9, high=10, size=n)
        result = linprog(c, A_eq=A, b_eq=b, method="revised simplex")
        if result.status == status:
            if status == 0 and u is not None and c.dot(u) < 0:
                raise Exception("u is cost decreasing!")
            if status == 3 and u is not None and c.dot(u) >= 0:
                raise Exception("u is not cost decreasing!", c, u, c.dot(u))
            break
        if i == max_iter - 1:
            raise Exception("max_iter reached.")

    return c

def certify_unbounded(c, u):
    x = np.ones(len(c), dtype=int)
    print(np.diff([c.dot(x + k*u) for k in range(10)]))


In [19]:
A, b, u = make_unbounded_polyhedron(8, 9)


157
Matrix([[9909120/46448411], [2216474/46448411], [46444295/46448411], [148538333/46448411], [16829964/46448411], [46000520/46448411], [95809363/46448411], [88141060/46448411], [1]])


In [20]:
c = make_c(A, b, u=u, status=0)
result = linprog(c, A_eq=A, b_eq=b, method="revised simplex")
print(result.x)

[0.93328914 0.9850781  0.68732452 0.         0.88669616 0.69031213
 0.3549856  0.40661068 0.68729681]


In [21]:
u = sympy.Matrix(A).nullspace()[0]
all([u[i, 0] > 0 for i in range(5)])

True

In [22]:
m, n = A.shape
bb = np.zeros(n, dtype=int)
bb[:m] = b.copy()
bb[m] = 0
I = np.identity(n, dtype=int)
for i in range(n):
    AA = np.zeros((n, n), dtype=int)
    AA[:m] = A.copy()
    AA[m] = I[i]
    x = np.linalg.solve(AA, bb)
    if np.all(x > 0):
        print(x)

[9.33289140e-01 9.85078101e-01 6.87324517e-01 5.12724772e-16
 8.86696157e-01 6.90312130e-01 3.54985605e-01 4.06610683e-01
 6.87296807e-01]


In [23]:
def make_polyhedron(m, n, rng=None, seed=None):
    if rng is None:
        rng = np.random.default_rng(seed)

    while True:
        A = rng.integers(-9, high=10, size=(m, n))
        _u, s, _vt = np.linalg.svd(A)
        if np.min(s) > 0.1:
            break
    
    b = A @ np.ones(n, dtype=int)
    s = np.sign(b + 0.1).astype(int)
    A *= s.reshape(-1, 1)
    b *= s
    
    return A, b


def subseqs(A, k):
    if k < 0 or len(A) < k:
        return []
    if k == 0:
        return [[]]
    ans = [[A[0], *s] for s in subseqs(A[1:], k-1)]
    ans.extend(subseqs(A[1:], k))
    return ans
    

def find_vertices(A, b):
    m, n = A.shape
    I = np.identity(n, dtype=int)
    S = subseqs(np.arange(n), n - m)
    xs = []
    for s in S:
        U = np.zeros((n, n+1), dtype=int)
        U[:m, :n] = A.copy()
        U[:m, n] = b.copy()
        U[m:, :n] = I[s]
        _u, s, _vt = np.linalg.svd(U)
        assert np.min(s) > 0.01
        V = sympy.Matrix(U)
        R = V.rref()[0]
        x = [R[i,n] for i in range(n)]
        if all(xi >= 0 for xi in x):
            xs.append(x)
    return xs

    

m, n = 5, 8
A, b = make_polyhedron(m, n)
vertices = find_vertices(A, b)
print(len(vertices))
V = np.array(vertices, dtype=float)
assert np.alltrue(V >= 0)
assert np.allclose(A @ V.T - b.reshape(-1, 1), 0)

12


In [24]:
v = V[0]
N = np.isclose(v, 0)
B = np.logical_not(N)
ABinv = np.linalg.inv(A[:,B])
x = np.zeros_like(v)
x[B] = ABinv @ b
assert np.allclose(v, x)

In [25]:
for j in [j for j in range(n) if N[j]]:
    d = np.zeros_like(v)
    d[j] = 1
    d[B] = -ABinv @ A[:,j]
    assert np.allclose(A @ d, 0)
    t = np.min(-x[d < 0]/d[d < 0])
    assert not np.isclose(t, 0)
    w = v + t*d
    assert np.alltrue(np.logical_or(np.isclose(w, 0), w > 0))
    assert np.allclose(A @ w, b)
    assert np.isclose(w, 0).sum() == n - m
    print(w)

[1.79800877e+00 1.70435370e+00 0.00000000e+00 2.57694904e+00
 0.00000000e+00 5.74097199e+00 2.22044605e-16 2.63415457e+00]
[0.00000000e+00 1.14140745e+00 1.07001530e+00 8.79602244e-01
 0.00000000e+00 9.99200722e-16 1.16251912e+00 8.50892402e-01]
[0.00000000e+00 1.13079966e+00 0.00000000e+00 4.37691187e-01
 3.36055991e-01 9.99200722e-16 1.09361136e+00 1.84709877e+00]


In [26]:
AA = sympy.Matrix(A)
bb = sympy.Matrix(b)
BB = [j for j in range(n) if B[j]]
AB = sympy.Matrix(m, m, lambda i, j: AA[i, BB[j]])
ABinv = AB.inv()
xB = ABinv @ bb
x = [0 for _ in range(n)]
for i in range(m):
    x[BB[i]] = xB[i]
x = sympy.Matrix(x)

In [27]:
adjacents = set()
for j in [j for j in range(n) if N[j]]:
    d = [0 for i in range(n)]
    d[j] = 1
    dd = -ABinv @ A[:,j]
    for i in range(m):
        d[BB[i]] = dd[i]
    d = sympy.Matrix(d)
    assert np.allclose(np.array(A @ sympy.Matrix(d), dtype=float), 0)
    t = min(*[-x[i]/d[i] for i in range(n) if d[i] < 0])
    assert t > 0
    w = x + t*d
    assert all([wi >= 0 for wi in w])
    assert AA @ w == bb
    assert len([i for i in range(n) if w[i] == 0]) == n - m
    w = sympy.DenseNDimArray(w)
    adjacents.add(w)
    display(w)

TypeError: 'Rational' object is not iterable

In [ ]:
adjacents

{[[0], [135/157], [8801/8007], [0], [3492/2669], [4360/2669], [3649/8007], [0]],
 [[0], [29673/39968], [82637/39968], [126671/39968], [22975/19984], [0], [0], [28945/39968]],
 [[81109/33450], [4717/33450], [6926/16725], [0], [0], [36656/16725], [0], [96493/16725]]}

In [31]:
def get_vertex(A, b, B):
    x = np.zeros(A.shape[1])
    x[B] = np.linalg.solve(A[:,B], b)
    return x

def get_adjacents(A, b, x):
    m, n = A.shape
    adjacents = []
    N = np.isclose(x, 0)
    B = np.logical_not(N)
    assert B.sum() == m
    U = np.linalg.inv(A[:, B])
    for j in [j for j in range(n) if N[j]]:
        d = np.zeros(n)
        d[j] = 1
        d[B] = - U @ A[:,j]
        assert np.allclose(A @ d, 0)
        neg_mask = np.logical_and(np.logical_not(np.isclose(d, 0)), d < 0)
        if np.any(neg_mask):
            t = np.min(-x[neg_mask]/d[neg_mask])
            assert not np.isclose(t, 0)
            y = x + t*d
            assert np.alltrue(np.logical_or(np.isclose(y, 0), y > 0))
            assert np.allclose(A @ y, b)
            assert np.isclose(y, 0).sum() == n - m
            adjacents.append(y)
    return adjacents

In [32]:
def subseqs(A, k):
    if k < 0 or len(A) < k:
        return []
    if k == 0:
        return [[]]
    ans = [[A[0], *s] for s in subseqs(A[1:], k-1)]
    ans.extend(subseqs(A[1:], k))
    return ans

def find_vertices(A, b):
    m, n = A.shape
    vertices = []
    for B in subseqs(np.arange(n), m):
        x = np.zeros(n)
        x[B] = np.linalg.solve(A[:,B], b)
        if np.alltrue(np.logical_or(np.isclose(x, 0), x > 0)):
            vertices.append(x)
    return vertices

In [33]:
m, n = 4, 8
A, b = make_polyhedron(m, n)
vertices = find_vertices(A, b)
x = vertices[0]

def hash_(x, n=1000):
    return hash(tuple(np.round(n*x).astype(int)))


visited = []
visitedh = set()
stack = [x]
for i in range(100):
    if stack == []:
        break
    x = stack.pop()
    xh = hash_(x)
    if xh not in visitedh:
        visited.append(x)
        visitedh.add(xh)
    adjacents = get_adjacents(A, b, x)
    for y in adjacents:
        yh = hash_(y)
        if yh not in visitedh:
            stack.append(y)
print(i, len(visited), len(vertices))

19 10 10


In [140]:
m, n = 4, 8
A, b = make_polyhedron(m, n)

A = sympy.Matrix(A)
b = sympy.Matrix(b)

In [56]:
def make_polyhedron(m, n, low=-9, high=10, rng=None, seed=None):
    if m > n:
        raise ValueError("Must have m <= n.")
    if rng is None:
        rng = np.random.default_rng(seed)
    B = list(range(m))
    while True:
        A = sympy.Matrix(rng.integers(low, high=high, size=(m, n)))
        if A[:, B].rank() == m:
            break
    x = sympy.Matrix([*[1 for i in range(m)], *[0 for i in range(n - m)]])
    b = A @ x
    for i in range(m):
        if b[i] < 0:
            A[i, :] = -A[i, :]
            b[i] = -b[i]
    return A, b


def subseqs(A, k):
    if k < 0 or len(A) < k:
        return []
    if k == 0:
        return [[]]
    ans = [[A[0], *s] for s in subseqs(A[1:], k-1)]
    ans.extend(subseqs(A[1:], k))
    return ans


def find_vertices(A, b):
    m, n = A.shape
    vertices = []
    for B in subseqs(list(range(n)), m):
        if A[:, B].rank() == m:
            x = sympy.matrices.zeros(n, 1)
            xB = A[:, B].inv() @ b
            feasible = True
            for i, xi in zip(B, xB):
                if xi < 0:
                    feasible = False
                    break
                x[i] = xi
            if feasible:
                vertices.append(sympy.DenseNDimArray(
                    [x[i, 0] for i in range(n)]))
    return vertices


def get_adjacents(A, x):
    m, n = A.shape
    adjacents = []
    B = [j for j in range(n) if x[j] != 0]
    N = [j for j in range(n) if x[j] == 0]
    assert len(B) == m and len(N) == n - m
    U = A[:, B].inv()
    for j in N:
        d = sympy.matrices.zeros(n, 1)
        d[j] = 1
        dB = - U @ A[:, j]
        for i, di in zip(B, dB):
            d[i] = di
        assert A @ d == sympy.matrices.zeros(m, 1)
        negs = [i for i in range(n) if d[i] < 0]
        if negs != []:
            t = min([-x[i]/d[i] for i in negs])
            y = sympy.Matrix(x) + t*d
            assert all([y[i] >= 0 for i in range(n)])
            assert len([i for i in range(n) if y[i] == 0]) >= n - m
            adjacents.append(sympy.DenseNDimArray([y[i] for i in range(n)]))
        else:
            print("Extreme!")
    return adjacents


def dfs(A, x, max_iter=1000):
    visited = set()
    stack = [x]
    for i in range(max_iter):
        if stack == []:
            break
        x = stack.pop()
        visited.add(x)
        adjacents = get_adjacents(A, x)
        for y in adjacents:
            if y not in visited:
                stack.append(y)
        if i == max_iter - 1:
            warn("max_iter reached.")
    return visited


In [58]:
m, n = 5, 10
rng = np.random.default_rng()
x = sympy.DenseNDimArray([*[1 for i in range(m)], *[0 for i in range(n - m)]])
for _ in range(10):
    A, b = make_polyhedron(m, n, rng=rng)
    vertices = find_vertices(A, b)
    visited = dfs(A, x)
    assert visited == set(vertices)

In [59]:
len(visited)

26

In [60]:
visited

{[0, 110028/77765, 0, 51744/77765, 310899/155530, 0, 50467/155530, 54334/77765, 0, 0],
 [0, 11031/23534, 50467/47068, 22465/23534, 48385/47068, 0, 0, 8037/23534, 0, 0],
 [0, 1187/11326, 9608/5663, 5193/11326, 0, 0, 0, 1807/11326, 0, 9677/11326],
 [0, 12266/43637, 130175/87274, 0, 0, 0, 6635/87274, 0, 8982/43637, 51251/43637],
 [0, 1437/2224, 785/1112, 617/1112, 1485/1112, 0, 0, 0, 893/2224, 0],
 [0, 14417/18064, 8815/9032, 0, 0, 6635/9032, 0, 0, 9419/18064, 10571/9032],
 [0, 1479/11168, 4535/2792, 1327/5584, 0, 0, 0, 0, 1807/11168, 2673/2792],
 [0, 17881/14962, 0, 4753/22443, 88085/44886, 0, 2355/14962, 0, 27167/44886, 0],
 [0, 1819/1408, 0, 9677/23232, 835/528, 2355/3872, 0, 0, 34543/46464, 0],
 [0, 2633/1396, 0, 0, 0, 695/349, 0, 205/698, 1171/1396, 939/698],
 [0, 27671/15247, 0, 22937/15247, 30037/30494, 50467/30494, 0, 34543/30494, 0, 0],
 [0, 3448/7347, 20641/14694, 0, 0, 0, 2885/14694, 1996/7347, 0, 9865/7347],
 [0, 37769/24128, 0, 0, 8815/12064, 15335/12064, 0, 0, 22083/24128, 9